###Spark Join exercise

####INNER JOIN

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
movie_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
                    .filter(f"file_date = '{v_file_date}'")

In [0]:
country_df = spark.read.format("delta").load(f"{silver_folder_path}/countries")

In [0]:
movies_companies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_companies")\
                    .filter(f"file_date = '{v_file_date}'")

In [0]:
production_countries_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_countries")\
                    .filter(f"file_date = '{v_file_date}'")

In [0]:
production_companies_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_companies")\
                    .filter(f"file_date = '{v_file_date}'")

In [0]:
movie_df.limit(10).display()

In [0]:
country_df.limit(10).display()

In [0]:
movies_companies_df.limit(10).display()

In [0]:
production_companies_df.limit(10).display()

In [0]:
production_countries_df.limit(10).display()

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_production_company_df = movies_companies_df.join(production_companies_df,
                                                        "company_id",
                                                        "inner") \
                                 .select(movies_companies_df.movie_id,production_companies_df.company_name,production_companies_df.company_id)

In [0]:
production_countries_df = country_df.join(production_countries_df,
                                 "country_id",
                                 "inner") \
                                 .select(production_countries_df.movie_id,country_df.country_id, country_df.country_name)

In [0]:
movie_filter_df = movie_df.filter("year_release_date >= 2010")

In [0]:
results_movie_genres_languages_df = movie_filter_df \
                                .join(production_countries_df,
                                      "movie_id",
                                      "inner") \
                                .join(movies_production_company_df,
                                      "movie_id",
                                      "inner") \
                                .select(movie_df.title,production_countries_df.country_id,production_countries_df.movie_id,movies_production_company_df.company_id,movie_df.duration_time, movie_df.release_date, movie_df.vote_average,production_countries_df.country_name,movies_production_company_df.company_name) \
                                .withColumn("created_date", lit(v_file_date)) \
                                .orderBy("title", ascending=True)

display(results_movie_genres_languages_df)

In [0]:
#overwrite_partition(results_movie_genres_languages_df,"movie_gold","results_country_prod_company","created_date")

In [0]:
#results_movie_genres_languages_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_movie_genres_languages")
#results_movie_genres_languages_df.write.mode("append").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_country_prod_company")
merge_condition='tgt.movie_id = src.movie_id and tgt.country_id = src.country_id and tgt.company_id = src.company_id and tgt.created_date = src.created_date'
merge_delta_lake(results_movie_genres_languages_df, "movie_gold", "results_country_prod_company", gold_folder_path, merge_condition, "created_date")

In [0]:
#results_movie_genres_languages_df.write.mode("append").partitionBy("created_date").parquet(f"{gold_folder_path}/results_movie_genres_languages")

In [0]:
%sql
select * from movie_gold.results_country_prod_company;

In [0]:
#display(spark.read.parquet(f"{gold_folder_path}/results_movie_genres_languages"))